In [33]:
import numpy as np
import tensorflow
import cv2
import time
import os
import math
import mediapipe as mp
from numpy import expand_dims , where ,argmax


os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [34]:
face_model = YOLO(r'c:/Users/dell/OneDrive/Desktop/Graduation project/yolov8n.pt') 
mobile_model = YOLO(r"c:/Users/dell/OneDrive/Desktop/Graduation project/best best (mobile) .pt")
names = mobile_model.model.names

In [35]:
# Define class index for 'person' (usually 0 in COCO dataset)
PERSON_CLASS_INDEX = 0

In [36]:
# Initialize video capture
cap = cv2.VideoCapture(0)  # Replace with your video path

In [37]:
# Frame count for tracking disappeared persons
time_disappear_limit = 10  # Number of frames to consider a person as exited # لو الشخص مختفي لمدة 30 فريم، نعتبره مشي

In [38]:
people_who_exist={}
session_start=0
session_end=0
exist_all_session=False
another_people=False
mobile=False
time_id = {}
time_temp = {}

# Eye State

In [39]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_CLOSED_THRESHOLD = 0.35

# Function to calculate EAR (Eye Aspect Ratio)
def calculate_ear(eye_landmarks):
    A = math.dist(eye_landmarks[1], eye_landmarks[5])
    B = math.dist(eye_landmarks[2], eye_landmarks[4])
    C = math.dist(eye_landmarks[0], eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_eyes_state(ROI):
   
    rows, cols, _ = ROI.shape
    posingEstimation = face_mesh.process(ROI)

    if posingEstimation.multi_face_landmarks:
        for facelms in posingEstimation.multi_face_landmarks:
            # Get key landmarks for gaze detection
            nose = facelms.landmark[1]  # Nose tip
            left_eye_outer = facelms.landmark[33]  # Left eye outer corner
            right_eye_outer = facelms.landmark[263]  # Right eye outer corner

            # Get landmarks for eye EAR calculation
            left_eye_landmarks = [
                facelms.landmark[i] for i in [33, 159, 158, 133, 153, 144]
            ]
            right_eye_landmarks = [
                facelms.landmark[i] for i in [362, 386, 385, 263, 373, 380]
            ]

           

            # Calculate EAR for both eyes
            left_eye_ear = calculate_ear(
                [(lm.x * cols, lm.y * rows) for lm in left_eye_landmarks]
            )
            right_eye_ear = calculate_ear(
                [(lm.x * cols, lm.y * rows) for lm in right_eye_landmarks]
            )

            # Determine if eyes are open or closed
            if left_eye_ear < EYE_CLOSED_THRESHOLD and right_eye_ear < EYE_CLOSED_THRESHOLD:
                eye_state = "Closed"
            else:
                eye_state = "Open"

            return  eye_state

    return "No Face Detected", "Unknown"



In [40]:
# calculate the time when the student closes his eye

def process_track(text, track_id):  
    if text == "Closed":
        if track_id not in time_temp:
            # If the track_id doesn't exist ------> add it with the current time 
            time_temp[track_id] = [int(time.time()) , True]

    elif text == "Open":
        # Calculate the difference between the current time and the previous eye_state
        if track_id in time_temp and time_temp[track_id][1]:
            current_time = int(time.time())
            time_difference = current_time - time_temp[track_id][0]
            # Update the time for the track_id
            if track_id in time_id:
                time_id[track_id] = max( time_difference , time_id[track_id])
            else:
                time_id[track_id] = time_difference
            time_temp.pop(track_id)
    

# Face recognition

In [41]:
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, \
MaxPooling2D, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.models import Model
def vgg_face():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    return model

In [42]:
model_vgg = vgg_face()
model_vgg = Model(inputs=model_vgg.layers[0].input, outputs=model_vgg.layers[-2].output)
model_vgg.load_weights('c:\\Users\\dell\\OneDrive\\Desktop\\Graduation project\\vgg_face_weights.h5')

d:\Anconda\envs\mediapipe_env\lib\site-packages\keras\src\layers\reshaping\zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [43]:
from tensorflow.keras.preprocessing.image import  img_to_array


# Preprocessing of image 
def preprocess_image(img):
    img = img_to_array(img)
    img = img/255.0
    img = expand_dims(img, axis=0)
    return img

# to identify the student
def Face_Recognition(roi):
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
    roi=cv2.resize(roi,dsize=(224,224),interpolation=cv2.INTER_CUBIC)
    roi=preprocess_image(roi)
    embedding_vector = model_vgg.predict(roi)[0]
    return embedding_vector


def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [44]:
import pandas as pd

# Create an empty DataFrame with 2623 columns
columns = [f"Column_{i}" for i in range(1, 2624)]
points_of_face = pd.DataFrame(columns=columns)

In [45]:
student_id_temp={}
student_id={}

In [46]:
# tracking
session_start=int(time.time())
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    face_recognized=set() # reset the set which contains faces that appear with each frame
    removed_ids=[]
    ###########################################################################################################################
    # Mobile_tracking
    result1=mobile_model(frame)

    for result in result1[0].boxes.cpu().numpy():
            x1,y1,x2,y2=map(int,result.xyxy[0])
            cls=names[int(result.cls[0])]
            conf=result.conf[0].round(2)
            
            if conf >0.90 :
                mobile=True
    
     
    ###########################################################################################################################
    # Run Face detection

    results = face_model.track(frame, persist=True, iou=0.5, show=False, tracker="bytetrack.yaml")  
    # Update existing persons and track new ones
    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()
        boxes = results[0].boxes.xyxy.cpu()
        classes = results[0].boxes.cls.int().cpu().tolist()
        confs = results[0].boxes.conf.tolist()
        
        detected_ids = set()

        for box, track_id, cls, conf in zip(boxes,track_ids, classes, confs):
            if cls == PERSON_CLASS_INDEX and conf > 0.4:  # Filter by class and confidence
                #####################################################################
                # eye state
                x1, y1, x2, y2 = box.int().tolist()   
                ROI = frame[y1:y2, x1:x2]
                if ROI.size != 0 :
                    ROI = cv2.cvtColor(ROI,cv2.COLOR_BGR2RGB)
                    text = detect_eyes_state(ROI)
                   
                   ###########################################################################
                    # face recognition

                    emded=Face_Recognition(ROI)
                    flag=False # define the flag to determine if the person appears for the first time or not
                    id=track_id
                    for i in range(points_of_face.shape[0]):
                        sim=findCosineSimilarity(emded,points_of_face.iloc[i,:-1])
                        if sim > 0.2 :
                            id=points_of_face.iloc[i,-1] # if person appear in the past -----> doesn't take new id
                            flag=True
                       
                        
                    face_recognized.add(id)   
                    if flag == False:
                        x=[] # list to collect vector of face_recognition (emded) and id 
                        for value in emded:
                            x.append(value) 
                        x.append(track_id)
                        
                        points_of_face.loc[len(points_of_face)] = (x)   # add new person to points_of_face dataframe
                    ##################################################################################  
                    if id not in time_id:
                        time_id[id] = 0
                    process_track(text, id)  
                    ################################################################################
                    # calculate time for each person appear
                    
                    if len(face_recognized)>0:
                        for i in face_recognized:
                            if i in student_id_temp.keys(): # if the person appears in the  frame (appears in face_recognized list) and he existed in previous frames ---> then he continues ---> then update last_seen_time only
                                student_id_temp[i]['last_seen_time']=int(time.time())
                            else: # he appears in the  frame (appears in face_recognized list) and he wasn't exist in previous frames -----> then he appears for the first time
                                student_id_temp[i]={
                                    'first_seen_time': int(time.time()),
                                    'last_seen_time': int(time.time())
                                }   

                    
                    if len(student_id_temp )>0 :
                        for j in student_id_temp.keys():  
                            if j not in face_recognized:# if the person exists in (student_id_temp) and not exists in (face_recognized) ----> then he is leave session ----> then I can calculate the duration that he appeared in it
                                if j not in student_id:
                                    student_id[j] = (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])
                                else:
                                    student_id[j] += (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])   

                                removed_ids.append(j) 

                    
                    for id in removed_ids: # remove the person that leaves from (student_id_temp) ------> bec if he appears again calculate time correctly by resetting (first_seen_time)
                        student_id_temp.pop(id,None) 
                            
                ###########################################################################
    
    # Show the frame for visualization (optional)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
cap.release()
cv2.destroyAllWindows()

session_end=int(time.time())


0: 480x640 (no detections), 212.7ms
Speed: 4.0ms preprocess, 212.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 75.0ms
Speed: 8.1ms preprocess, 75.0ms inference, 7.2ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step

0: 480x640 (no detections), 321.4ms
Speed: 6.5ms preprocess, 321.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 76.6ms
Speed: 3.3ms preprocess, 76.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step

0: 480x640 1 phone, 203.6ms
Speed: 1.0ms preprocess, 203.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 76.2ms
Speed: 3.1ms preprocess, 76.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step

0: 480x640 1 phone, 231.3ms
Speed: 2.5ms preprocess, 231.3ms inference, 0.0ms postprocess per image at shap

# preprocessing before calculation

In [47]:
# calculate the time for people who exist until the end
for j in student_id_temp.keys():
    if j not in student_id:
        student_id[j] = (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])
    else:
        student_id[j] += (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time']) 

In [48]:
student_id

{1: 12}

In [49]:
total_time_of_session=session_end-session_start # calculate the total time of the session from beginning to end
upper_time_limit=total_time_of_session*0.95 # the maximum limit allowed
lower_time_limit=total_time_of_session*0.90 # the minimum limit allowed
limit_of_another_people=total_time_of_session*0.10 # limit of appearance of another person during the session is 10% of the total session time
limit_of_closed_eyes=total_time_of_session*0.15 # limit of closing your eyes is 15% of the total session time

In [50]:
total_score=100 # Attention_&_focussing rate

In [51]:
# the real student -----> who is more people attended
student_duration=0
for key,value in student_id.items():
    if value >= student_duration:
        student_duration=value
        id_of_real_student=key

In [52]:
print('total people appear',student_id)

total people appear {1: 12}


# calculation

In [53]:
 # studend_id ----> contain {ID of person , duration of appearance}
if (total_time_of_session - student_duration) >= upper_time_limit : # time that the student doesn't exist in session more than 5% of the total session time 
    total_score-=20
if  (total_time_of_session - student_duration) > lower_time_limit: # if the student doesn't attend more than 10% of total session time (try to cover the camera) -----> then he is absent
    total_score-=100   

In [54]:
for i,j in student_id.items():
    if( i != id_of_real_student):
        if ((j >= limit_of_another_people) and len(student_id) ) > 1: # if there is another person and exists more than the allowd_time(10% of total session time)
            total_score-=20
            break

In [55]:
if mobile:  # if a mobile phone appears with the student -----> it will decrease the focussing rate by 40%
  total_score-=40


In [56]:
if time_id[id_of_real_student] > limit_of_closed_eyes: # if the student was sleep during session 
   total_score-=20   

In [57]:
total_score=max(0,total_score)
percentage_of_attention=(total_score)/100

In [58]:
percentage_of_attention

1.0

# Report

In [59]:
# if the student holds the mobile phone or not
mobile

False

In [60]:
# how many time the student sleeps during the session
time_id[id_of_real_student]

1

In [61]:
# how many time the student doesn't attend the session or try to covers the camera
(total_time_of_session - student_duration)

3

In [62]:
# how many another people exist during the session and how many time they attend
count_of_another_people=0
time_of_another_people=0


for i,j in student_id.items():
    if( i != id_of_real_student):
       count_of_another_people +=1
       time_of_another_people += j


print(count_of_another_people)
print(time_of_another_people)

0
0
